<img src="https://github.com/richardcerny/bricksflow/raw/rc-bricksflow2.1/docs/img/databricks_icon.png?raw=true" width=100/>
# Bricksflow example 5.

Widgets, secrets, notebookFunction

## Widgets
Many people love widgets as they can easilly parametrize their notebook. It is possible to use widget with Bricksflow. Usage is demonstrated in this notebook. Don't forget to check [Widgets documentation](ttps://docs.databricks.com/notebooks/widgets.html) or run command `dbutils.widgets.help()` to see options you have while working with widget.

<img src="https://github.com/richardcerny/bricksflow/raw/rc-bricksflow2.1/docs/img/widgets.PNG?raw=true" width=1000/>

In [0]:
%run ../../../app/install_master_package

Library with this file name dbfs:/FileStore/jars/t__myproject__t/2021-01-13_14-36-37_nvrfpjmnea/t__myproject__t-0.1-py3-none-any.whl already installed. If you would like to reinstall this, please detach and re-attach the notebook to recreate your environment. Conflict with previous lib file: dbfs:/FileStore/jars/t__myproject__t/2021-01-13_14-36-37_nvrfpjmnea/t__myproject__t-0.1-py3-none-any.whl
Out[16]: False

In [0]:
from logging import Logger
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from databricksbundle.notebook.decorators import dataFrameLoader, transformation, dataFrameSaver, notebookFunction
from datalakebundle.table.TableNames import TableNames

from pyspark.sql import functions as F
from pyspark.dbutils import DBUtils # enables to use Datbricks dbutils within functions

### Create a widget

In [0]:
@notebookFunction()
def create_input_widgets(dbutils: DBUtils):
  dbutils.widgets.text("widget_states", "AL", 'States') # Examples: CA, IL, IN,...
  dbutils.widgets.dropdown("widget_year", '2020', ['2018', '2019', '2020'], 'Example Dropdown Widget') # Examples: CA, IL, IN,...

### Usage of widget variable

In [0]:
@dataFrameLoader(display=False)
def read_bronze_covid_tbl_template_2_confirmed_case(spark: SparkSession, logger: Logger, tableNames: TableNames, dbutils: DBUtils):
    stateName = dbutils.widgets.get("widget_states")
    logger.info(f"States Widget value: {stateName}")
    return (
        spark
            .read
            .table(tableNames.getByAlias('bronze_covid.tbl_template_2_confirmed_cases'))
            .select('countyFIPS','County_Name', 'State', 'stateFIPS')
            .filter(F.col('State')==stateName) # Widget variable is used
    )

15:01:27 INFO - States Widget value: AL
{dbName: dev_gold_solutions, tableIdentifier: template_5_solution, dbIdentifier: gold_solutions, tableName: template_5_solution}

In [0]:
@transformation(read_bronze_covid_tbl_template_2_confirmed_case, display=True)
def add_year_value(df: DataFrame, logger: Logger, dbutils: DBUtils):
    yearWidget = dbutils.widgets.get("widget_year")
    logger.info(f"Year Widget value: {yearWidget}")
    return (
        df
          .withColumn('WIDGET_YEAR', F.lit(yearWidget))
    )

countyFIPS,County_Name,State,stateFIPS,WIDGET_YEAR
0,Statewide Unallocated,AL,1,2020
1001,Autauga County,AL,1,2020
1003,Baldwin County,AL,1,2020
1005,Barbour County,AL,1,2020
1007,Bibb County,AL,1,2020
1009,Blount County,AL,1,2020
1011,Bullock County,AL,1,2020
1013,Butler County,AL,1,2020
1015,Calhoun County,AL,1,2020
1017,Chambers County,AL,1,2020


### Work with Pandas

In [0]:
from pandas.core.frame import DataFrame as pdDataFrame

In [0]:
@notebookFunction(add_year_value)
def spark_df_to_pandas(df: DataFrame) -> pdDataFrame:
    return df.toPandas()

In [0]:
type(spark_df_to_pandas.result)

Out[24]: pandas.core.frame.DataFrame

In [0]:
@notebookFunction(spark_df_to_pandas.result)
def pandasTranformation(pd: pdDataFrame):
    pd2 = pd['WIDGET_YEAR']
    print(pd2)
    return pd2

0 2020
1 2020
2 2020
3 2020
4 2020
5 2020
6 2020
7 2020
8 2020
9 2020
Name: WIDGET_YEAR, dtype: object

### Pass notebookFunction to other tranformations
See `other_python_function.result`

In [0]:
@notebookFunction()
def other_python_function(dbutils: DBUtils):
    return dbutils.widgets.get("widget_states")

In [0]:
@transformation(read_bronze_covid_tbl_template_2_confirmed_case, other_python_function.result, display=True)
def add_year_value_2(df: DataFrame, widget_test: str, logger: Logger):
    yearWidget = widget_test
    logger.info(f"Year Widget value: {yearWidget}")
    return (
        df
          .withColumn('other_python_function_VALUE', F.lit(yearWidget))
    )

countyFIPS,County_Name,State,stateFIPS,other_python_function_VALUE
0,Statewide Unallocated,AL,1,AL
1001,Autauga County,AL,1,AL
1003,Baldwin County,AL,1,AL
1005,Barbour County,AL,1,AL
1007,Bibb County,AL,1,AL
1009,Blount County,AL,1,AL
1011,Bullock County,AL,1,AL
1013,Butler County,AL,1,AL
1015,Calhoun County,AL,1,AL
1017,Chambers County,AL,1,AL


### Command to remove all Widget

In [0]:
#dbutils.widgets.removeAll()